<a href="https://colab.research.google.com/github/PalomaAlmendros/03MAIR---Algoritmos-de-Optimizacion---2019/blob/master/Paloma_Almendros_AG2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* AG2- Algoritmos de Optimización
* Paloma Almendros Jimenez
* https://colab.research.google.com/drive/1ZGJ3PVjTdp2Ce6hhuQw99dzwbis1sJFl

*** EJERCICIO1 ***
Programación dinámica
Calcular la ruta más rápida y económica para ir de un punto a otro

In [0]:
TARIFAS = [
[0,5,4,3,999,999,999],
[999,0,999,2,3,999,11],
[999,999, 0,1,999,4,10],
[999,999,999, 0,5,6,9],
[999,999, 999,999,0,999,4],
[999,999, 999,999,999,0,3],
[999,999,999,999,999,999,0]
]
# Los precios altos son para rellenar huecos de la matriz, y con ese valor el programa nunca los va a dar por óptimos

In [0]:
def Precios(TARIFAS):
  N = len(TARIFAS[0])                       # N es el número de nodos
  
  #Inicialización de la tabla de precios
  PRECIOS = [ [999]*N for i in [999]*N]
  RUTA = [ [""]*N for i in [""]*N]
  
  for i in range(N-1):                      # no necesito recorrer hasta el N (no me interesa el N-N), me sobra con quedarme en el anterior
    for j in range(i+1, N):                 # pongo i+1 porque el recorrido 0-0 no me interesa tampoco
      MIN = TARIFAS[i][j]
      RUTA[i][j] = i
      
      for k in range(i, j):                 # usamos k para puntos intermedios
        if PRECIOS[i][k] + TARIFAS[k][j] < MIN:
            MIN = min(MIN, PRECIOS[i][k] + TARIFAS[k][j] )
            RUTA[i][j] = k
        PRECIOS[i][j] = MIN
        
  return PRECIOS,RUTA

In [0]:
PRECIOS,RUTA = Precios(TARIFAS)  
#print(PRECIOS[0][6])
print("PRECIOS")
for i in range(len(TARIFAS)):
  print(PRECIOS[i])

print("\nRUTA")  
for i in range(len(TARIFAS)):
  print(RUTA[i]) 

PRECIOS
[999, 5, 4, 3, 8, 8, 11]
[999, 999, 999, 2, 3, 8, 7]
[999, 999, 999, 1, 6, 4, 7]
[999, 999, 999, 999, 5, 6, 9]
[999, 999, 999, 999, 999, 999, 4]
[999, 999, 999, 999, 999, 999, 3]
[999, 999, 999, 999, 999, 999, 999]

RUTA
['', 0, 0, 0, 1, 2, 5]
['', '', 1, 1, 1, 3, 4]
['', '', '', 2, 3, 2, 5]
['', '', '', '', 3, 3, 3]
['', '', '', '', '', 4, 4]
['', '', '', '', '', '', 5]
['', '', '', '', '', '', '']


In [0]:
def calcular_ruta(RUTA, desde, hasta):
  if desde == hasta:
    return desde
  else:
    return str(calcular_ruta(RUTA, desde, RUTA[desde][hasta])) +  ',' + str(RUTA[desde][hasta])
    
print('\nLa ruta más rápida y económica es pasando por estos puntos: ')
calcular_ruta(RUTA,0,6) 


La ruta más rápida y económica es pasando por estos puntos: 


'0,0,2,5'

****EJERCICIO 2 ****
Ramificación y Poda
Problema: Asignación de 4 tareas a 4 agentes

In [0]:
COSTES=[[11,12,18,40],
        [14,15,13,22],
        [11,17,19,23],
        [17,14,20,28]]

In [0]:
def valor(S,COSTES):
  VALOR = 0
  for i in range(len(S)):
    VALOR += COSTES[S[i]][i] 
  return VALOR
  
valor((0, 1, 2, 3),COSTES)

73

In [0]:
# Coste inferior para soluciones parciales
# (1, 3, ) Se asigna la tarea 1 al agente 0 y la tarea 3 al agente 1

def CI(S,COSTES):
  VALOR = 0
  #Valores establecidos
  for i in range(len(S)):
    VALOR += COSTES[i][S[i]]

  #Estimacion
  for i in range(len(COSTES)):
    if i not in S:
      VALOR += min([COSTES[j][i] for j in range(len(S), len(COSTES))])
  return VALOR

def CS(S,COSTES):
  VALOR = 0
  #Valores establecidos
  for i in range(len(S)):
    VALOR += COSTES[i][S[i]]

  #Estimacion
  for i in range(len(COSTES)):
    if i not in S:
      VALOR += max( [ COSTES[j][i] for j in range(len(S), len(COSTES))])
  return VALOR

In [0]:
#Genera tantos hijos como posibilidades haya para el siguiente elemento de la tupla
#(0,) -> (0,1), (0,2), (0,3)
def crear_hijos(NODO, N):
  HIJOS = []
  for i in range(N):
    if i not in NODO:
      HIJOS.append({'s':NODO +(i,)})
  return HIJOS

In [0]:
def ramificacion_y_poda(COSTES):
#Construccion iterativa de soluciones(arbol). En cada etapa asignamos un agente(ramas).
#Nodos del grafo  {s:(1,2),CI:3,CS:5}
  
  DIMENSION = len(COSTES)  
  MEJOR_SOLUCION=tuple(i for i in range(len(COSTES)))
  CotaSup = valor(MEJOR_SOLUCION,COSTES)
  
  NODOS=[]
  NODOS.append({'s':(), 'ci':CI((), COSTES)})

  iteracion = 0

  while(len(NODOS) > 0):
    iteracion +=1

    nodo_prometedor = [ min(NODOS, key=lambda x:x['ci']) ][0]['s']
   
    #Ramificacion
    #Se generan los hijos
    HIJOS =[ {'s':x['s'], 'ci':CI(x['s'], COSTES)   } for x in crear_hijos(nodo_prometedor, DIMENSION) ]

    #Revisamos la cota superior y nos quedamos con la mejor solución si llegamos a una solución final
    NODO_FINAL = [x for x in HIJOS if len(x['s']) == DIMENSION]
    if len(NODO_FINAL) > 0: 
      if NODO_FINAL[0]['ci'] < CotaSup:
        CotaSup = NODO_FINAL[0]['ci']
        MEJOR_SOLUCION = NODO_FINAL
 
    #Poda
    HIJOS = [x for x in HIJOS if x['ci'] < CotaSup]

    #Añadimos los hijos 
    NODOS.extend(HIJOS) 

    #Eliminamos el nodo ramificado
    NODOS =  [x for x in NODOS if x['s'] != nodo_prometedor]
   
    print("La solucion final es:" ,MEJOR_SOLUCION , " en " , iteracion , " iteraciones" , " para dimension: " ,DIMENSION  )
      


ramificacion_y_poda(COSTES)


La solucion final es: (0, 1, 2, 3)  en  1  iteraciones  para dimension:  4
La solucion final es: (0, 1, 2, 3)  en  2  iteraciones  para dimension:  4
La solucion final es: (0, 1, 2, 3)  en  3  iteraciones  para dimension:  4
La solucion final es: (0, 1, 2, 3)  en  4  iteraciones  para dimension:  4
La solucion final es: (0, 1, 2, 3)  en  5  iteraciones  para dimension:  4
La solucion final es: [{'s': (0, 2, 3, 1), 'ci': 61}]  en  6  iteraciones  para dimension:  4
La solucion final es: [{'s': (0, 2, 3, 1), 'ci': 61}]  en  7  iteraciones  para dimension:  4
La solucion final es: [{'s': (0, 2, 3, 1), 'ci': 61}]  en  8  iteraciones  para dimension:  4
La solucion final es: [{'s': (0, 2, 3, 1), 'ci': 61}]  en  9  iteraciones  para dimension:  4
La solucion final es: [{'s': (0, 2, 3, 1), 'ci': 61}]  en  10  iteraciones  para dimension:  4
La solucion final es: [{'s': (0, 2, 3, 1), 'ci': 61}]  en  11  iteraciones  para dimension:  4
La solucion final es: [{'s': (0, 2, 3, 1), 'ci': 61}]  en  

* Para matrices mayores, por ejemplo de dimensión 8

In [0]:
COSTES8=[[11,12,18,40,34,56,78,12],
        [14,15,13,22,45,65,76,23],
        [11,17,19,23,43,23,67,89],
        [17,14,20,28,33,55,88,33],
        [11,23,45,75,87,77,67,56],
        [23,69,74,52,56,80,86,51],
        [99,87,78,98,65,73,25,66],
        [11,22,33,44,55,66,77,88]]

In [0]:
def valor(S,COSTES8):
  VALOR = 0
  for i in range(len(S)):
    VALOR += COSTES8[S[i]][i] 
  return VALOR
  
valor((0, 1, 2, 3, 4, 5, 6, 7),COSTES8)

353

In [0]:
# Coste inferior para soluciones parciales
# (1, 3, ) Se asigna la tarea 1 al agente 0 y la tarea 3 al agente 1

def CI(S,COSTES8):
  VALOR = 0
  #Valores establecidos
  for i in range(len(S)):
    VALOR += COSTES8[i][S[i]]

  #Estimacion
  for i in range(len(COSTES8)):
    if i not in S:
      VALOR += min([COSTES8[j][i] for j in range(len(S), len(COSTES8))])
  return VALOR

def CS(S,COSTES8):
  VALOR = 0
  #Valores establecidos
  for i in range(len(S)):
    VALOR += COSTES8[i][S[i]]

  #Estimacion
  for i in range(len(COSTES8)):
    if i not in S:
      VALOR += max( [ COSTES8[j][i] for j in range(len(S), len(COSTES8))])
  return VALOR

In [0]:
#Genera tantos hijos como posibilidades haya para el siguiente elemento de la tupla
#(0,) -> (0,1), (0,2), (0,3)
def crear_hijos(NODO, N):
  HIJOS = []
  for i in range(N):
    if i not in NODO:
      HIJOS.append({'s':NODO +(i,)})
  return HIJOS

In [0]:
def ramificacion_y_poda(COSTES8):
#Construccion iterativa de soluciones(arbol). En cada etapa asignamos un agente(ramas).
#Nodos del grafo  {s:(1,2),CI:3,CS:5}
  
  DIMENSION = len(COSTES8)  
  MEJOR_SOLUCION=tuple(i for i in range(len(COSTES8)))
  CotaSup = valor(MEJOR_SOLUCION,COSTES8)
  
  NODOS=[]
  NODOS.append({'s':(), 'ci':CI((), COSTES8)})

  iteracion = 0

  while(len(NODOS) > 0):
    iteracion +=1

    nodo_prometedor = [ min(NODOS, key=lambda x:x['ci']) ][0]['s']
   
    #Ramificacion
    #Se generan los hijos
    HIJOS =[ {'s':x['s'], 'ci':CI(x['s'], COSTES8)   } for x in crear_hijos(nodo_prometedor, DIMENSION) ]

    #Revisamos la cota superior y nos quedamos con la mejor solución si llegamos a una solución final
    NODO_FINAL = [x for x in HIJOS if len(x['s']) == DIMENSION]
    if len(NODO_FINAL) > 0: 
      if NODO_FINAL[0]['ci'] < CotaSup:
        CotaSup = NODO_FINAL[0]['ci']
        MEJOR_SOLUCION = NODO_FINAL
 
    #Poda
    HIJOS = [x for x in HIJOS if x['ci'] < CotaSup]

    #Añadimos los hijos 
    NODOS.extend(HIJOS) 

    #Eliminamos el nodo ramificado
    NODOS =  [x for x in NODOS if x['s'] != nodo_prometedor]
   
    print("La solucion final es:" ,MEJOR_SOLUCION , " en " , iteracion , " iteraciones" , " para dimension: " ,DIMENSION  )
      


ramificacion_y_poda(COSTES8)

La solucion final es: (0, 1, 2, 3, 4, 5, 6, 7)  en  1  iteraciones  para dimension:  8
La solucion final es: (0, 1, 2, 3, 4, 5, 6, 7)  en  2  iteraciones  para dimension:  8
La solucion final es: (0, 1, 2, 3, 4, 5, 6, 7)  en  3  iteraciones  para dimension:  8
La solucion final es: (0, 1, 2, 3, 4, 5, 6, 7)  en  4  iteraciones  para dimension:  8
La solucion final es: (0, 1, 2, 3, 4, 5, 6, 7)  en  5  iteraciones  para dimension:  8
La solucion final es: (0, 1, 2, 3, 4, 5, 6, 7)  en  6  iteraciones  para dimension:  8
La solucion final es: (0, 1, 2, 3, 4, 5, 6, 7)  en  7  iteraciones  para dimension:  8
La solucion final es: (0, 1, 2, 3, 4, 5, 6, 7)  en  8  iteraciones  para dimension:  8
La solucion final es: (0, 1, 2, 3, 4, 5, 6, 7)  en  9  iteraciones  para dimension:  8
La solucion final es: (0, 1, 2, 3, 4, 5, 6, 7)  en  10  iteraciones  para dimension:  8
La solucion final es: (0, 1, 2, 3, 4, 5, 6, 7)  en  11  iteraciones  para dimension:  8
La solucion final es: (0, 1, 2, 3, 4, 5, 